In [29]:
import tensorflow as tf
import numpy as np

In [2]:
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b
with tf.Session() as sess:
    print(sess.run(c))
    print(c.eval())

30.0
30.0


In [4]:
W1 = tf.ones((2, 2))
W2 = tf.Variable(tf.zeros((2, 2)), name='weights')

with tf.Session() as sess:
    print(sess.run(W1))
    sess.run(tf.global_variables_initializer())
    print(sess.run(W2))

[[ 1.  1.]
 [ 1.  1.]]
[[ 0.  0.]
 [ 0.  0.]]


In [9]:
state = tf.Variable(0, name='counter')
new_value = tf.add(state, tf.constant(1))
update = tf.assign(state, new_value)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(state))
    for _ in range(5):
          value = sess.run([update, new_value])
          print(value)

0
[1, 1]
[2, 2]
[3, 3]
[4, 4]
[5, 5]


In [19]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = input1 * input2

with tf.Session() as sess:
    result = sess.run([output], feed_dict={input1:7.0, input2:2.0})
    print(result)

[14.0]


In [28]:
with tf.variable_scope('foo', reuse=False) as scope:
    #scope.reuse_variables()
    v = tf.get_variable('v2', [1, 2])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    result = sess.run([v])
    print(result)

[array([[ 0.54536271,  1.19627726]], dtype=float32)]


In [72]:
batch_size = 10
num_unrollings = 5
vocabulary_size = 7

model = tf.Graph()
with model.as_default():
    def zeros(shape):
        initial_value = tf.zeros(shape)
        return tf.Variable(initial_value, trainable=False)
    
    saved_output = zeros((batch_size, vocabulary_size))
    saved_state = zeros((batch_size, vocabulary_size))
    
    def cell(i, o, s):
        return (i + o, i + s + 1)
    
    inputs = []
    for _ in range(num_unrollings):
        placeholder = tf.placeholder(tf.float32,
                                     shape=(batch_size, vocabulary_size))
        inputs.append(placeholder)
    
    o = saved_output
    s = saved_state
    outputs = []
    for value in inputs:
        (o, s) = cell(value, o, s)
        outputs.append(o)
    
    #dependency = [saved_output.assign(o),
    #              saved_state.assign(s)]
    #with tf.control_dependencies(dependency):
    x = tf.concat(outputs, 0)
    y = tf.reduce_mean(x)

In [73]:
n_iters = 10

with tf.Session(graph=model) as session:
    session.run(tf.global_variables_initializer())
    for _ in range(n_iters):
        feed_dict = {}
        for i in range(num_unrollings):
            ith_batch = np.zeros((batch_size, vocabulary_size))
            ith_batch[:, i] = i
            feed_dict[inputs[i]] = ith_batch

        result = session.run([o, s, y], feed_dict=feed_dict)
        print('Outputs')
        print(result[0])
        print('State')
        print(result[1])
        print('Mean')
        print(result[2])
        print('################# ###################### #############')

Outputs
[[ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]]
State
[[ 5.  6.  7.  8.  9.  5.  5.]
 [ 5.  6.  7.  8.  9.  5.  5.]
 [ 5.  6.  7.  8.  9.  5.  5.]
 [ 5.  6.  7.  8.  9.  5.  5.]
 [ 5.  6.  7.  8.  9.  5.  5.]
 [ 5.  6.  7.  8.  9.  5.  5.]
 [ 5.  6.  7.  8.  9.  5.  5.]
 [ 5.  6.  7.  8.  9.  5.  5.]
 [ 5.  6.  7.  8.  9.  5.  5.]
 [ 5.  6.  7.  8.  9.  5.  5.]]
Mean
0.571429
################# ###################### #############
Outputs
[[ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1.  2.  3.  4.  0.  0.]
 [ 0.  1

In [76]:
a = ()
for i in (1, 2, 3):
    a += ([i], )
    
a

([1], [2], [3])